<h1 style="text-align:center; color:purple">House price prediction using Linear regression</h1>

<h2 style="text-align:center; color:pink">Presentation</h2>

<center>
    djhedbezdde
</center>

<h2>1 - Installations and Imports</h2>

In [8]:
%%capture
!pip3 install numpy
!pip3 install matplotlib
!pip3 install pandas
!pip3 install scipy
from scipy.stats import norm

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy
import math
%matplotlib inline

<h2>2 - Load dataset inside the file</h2>

In [11]:
df = pd.read_csv('dataset-houseprice2.csv')

In [12]:
df.head()

,price,bedrooms,bathrooms,sqft,floors,condition,grade,yr_built
0,221900.0,3,1.00,1180,1.0,3,7,1955
1,538000.0,3,2.25,2570,2.0,3,7,1951
2,180000.0,2,1.00,770,1.0,3,6,1933
3,604000.0,4,3.00,1960,1.0,5,7,1965
4,510000.0,3,2.00,1680,1.0,3,8,1987
